In [2]:
#required for manipulating data
import pandas as pd
import numpy as np
#required for loading google APIs
import gspread
#required for building interactive dashboard
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

Connect to your bank data

In [3]:
#connect to google drive
gc = gspread.service_account(filename= 'service_account.json')
ft = gc.open('femi_transaction')
#note that you can also connect to the file if it is on your pc by using df = pd.read_csv('filename.csv')

In [4]:
ws = ft.worksheet('UBA')
df = pd.DataFrame(ws.get_all_records())
df.head()


,Transaction Date,Description,Value Date,Debit,Credit,Currency,Balance after Transaction
0,01-Jan-2022,SBW/PATU/1149225362/2136928210/2349024484050,02-Jan-2022,300.0,0.0,NGN,4122.87
1,01-Jan-2022,SBW/PATU/1149374657/2136928210/2349024484050,02-Jan-2022,100.0,0.0,NGN,4022.87
2,03-Jan-2022,POS Pur @ 2KUD1DYL-LA LA,03-Jan-2022,2000.0,0.0,NGN,2022.87
3,05-Jan-2022,POS Pur @ 20570P9M-LA LA,05-Jan-2022,2000.0,0.0,NGN,22.87
4,09-Jan-2022,NXG :TRFFRMOJOAYODEJIIWALOYETOFEMINOAHOJ,08-Jan-2022,0.0,14000.0,NGN,14022.87


In [5]:
#clean the data
df = df[['Transaction Date', 'Description', 'Debit', 'Credit', 'Currency', 'Balance after Transaction']] #this keeps only the desired columns
df= df.reset_index(drop=True)
df['Description'] = df['Description'].map(str.lower) #turns all the string in description to lower case
df = df.rename(columns={'Transaction Date' : 'Date'}) # i rename the transaction date into date
df['Category'] = 'Unassigned' #this add a new column called category 
df.head()

,Date,Description,Debit,Credit,Currency,Balance after Transaction,Category
0,01-Jan-2022,sbw/patu/1149225362/2136928210/2349024484050,300.0,0.0,NGN,4122.87,Unassigned
1,01-Jan-2022,sbw/patu/1149374657/2136928210/2349024484050,100.0,0.0,NGN,4022.87,Unassigned
2,03-Jan-2022,pos pur @ 2kud1dyl-la la,2000.0,0.0,NGN,2022.87,Unassigned
3,05-Jan-2022,pos pur @ 20570p9m-la la,2000.0,0.0,NGN,22.87,Unassigned
4,09-Jan-2022,nxg :trffrmojoayodejiiwaloyetofeminoahoj,0.0,14000.0,NGN,14022.87,Unassigned


In [6]:
del df['Credit']

In [7]:
#define categories
#Self-care/Services
#Giving
#Airtime
#Bank-Charges
#Gov-Charges
#Entertainment

In [7]:
# Entertainment

df['Category'] = np.where(df['Description'].str.contains('web',case=False), 'Entertainment', df['Category'])

# Bank-Charges

df['Category'] = np.where(df['Description'].str.contains('maint|wht|charges|sms|cif', case=False), 'Bank-Charges', df['Category'])

# Airtime

df['Category'] = np.where(df['Description'].str.contains('topup|patu|satu|catu|aatu',case=False), 'Airtime', df['Category'])

# Self-care/Services

df['Category'] = np.where(df['Description'].str.contains('pos|cwb|wd|withd|cwd', case=False), 'Self-care/Services', df['Category'])

# Gov-charges

df['Category'] = np.where(df['Description'].str.contains('fgn',case=False), 'Gov-charges', df['Category'])

# Gift

df['Category'] = np.where(df['Description'].str.contains('utu|uto|to|utp', case=False)  & df['Debit'] != 0, 'Giving', df['Category'])
df['Category'] = np.where(df['Debit'] == 0, 'Credit', df['Category'])


# Change the date columm to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

#Extract month and year
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year                        
pd.options.display.max_rows = 999
df.head()

,Date,Description,Debit,Currency,Balance after Transaction,Category,Month,Year
0,2022-01-01,sbw/patu/1149225362/2136928210/2349024484050,300.0,NGN,4122.87,Airtime,1.0,2022.0
1,2022-01-01,sbw/patu/1149374657/2136928210/2349024484050,100.0,NGN,4022.87,Airtime,1.0,2022.0
2,2022-01-03,pos pur @ 2kud1dyl-la la,2000.0,NGN,2022.87,Self-care/Services,1.0,2022.0
3,2022-01-05,pos pur @ 20570p9m-la la,2000.0,NGN,22.87,Self-care/Services,1.0,2022.0
4,2022-01-09,nxg :trffrmojoayodejiiwaloyetofeminoahoj,0.0,NGN,14022.87,Credit,1.0,2022.0


In [9]:
# This code checks unassigned transactions and confirm all transactions are assigned to a category
unassigned = df.loc[df['Category'] == 'unassigned']
if unassigned.empty:
    raise ValueError("No 'unassigned' records found in the DataFrame. This is good")
else:
    print(unassigned)

ValueError: No 'unassigned' records found in the DataFrame. This is good

*<font color='blue'><b> Create Top Banner for a summary of last month's income, recurring expenses, non-recurring expenses and savings
</font>*

In [9]:
# I used the following code to get the latest month and year
latest_month = df['Month'].max()
latest_year = df['Year'].max()

# This code is used to filter the dataframe to include only transactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]

In [10]:
last_month_expenses = last_month_expenses.groupby('Category')['Debit'].sum().reset_index()

last_month_expenses['Debit']=last_month_expenses['Debit'].astype('str')
last_month_expenses['Debit']=last_month_expenses['Debit'].str.replace('-','')
last_month_expenses['Debit']=last_month_expenses['Debit'].astype('float')        #get absolute figures
last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Credit") == False]    #exclude "credit" category

last_month_expenses = last_month_expenses.sort_values(by='Debit', ascending=False)    #sort values
last_month_expenses['Debit'] = last_month_expenses['Debit'].round().astype(int)      #round values

last_month_expenses

,Category,Debit
4,Self-care/Services,14400
2,Giving,8022
3,Gov-charges,50
0,Bank-Charges,40


In [11]:
last_month_expenses_total = last_month_expenses['Debit'].sum()
last_month_expenses_total

22512

In [12]:
def calculate_difference(event):
    """
    Calculate the financial difference based on income, recurring expenses, and monthly expenses.

    Parameters:
    - event: The event triggering the calculation.

    This function takes input values from specified widgets (income_widget, recurring_expenses_widget,
    monthly_expenses_widget), calculates the financial difference, and updates the difference in the
    difference_widget.

    Note:
    - Ensure that the input widgets (income_widget, recurring_expenses_widget, monthly_expenses_widget)
      are appropriately defined and accessible within the scope of this function.

    Example Usage:
    calculate_difference('button_click_event')

    """
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses #calculates the difference
    difference_widget.value = str(difference) #retrieves the difference_widget_value

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_total))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")
# The lines of code below add event listeners to the input widget using the param watch method
income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")
pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()


Launching server at http://localhost:52801


*<font color= 'blue' ><b>Last Month Expenses Bar Chart</b> <font/>* 

In [13]:
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Debit', 
    height=250, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 15000))

last_month_expenses_chart



:Bars   [Category]   (Debit)

*<font color='blue'><b> Expenses by Month Summary<b/> <font/>* 

In [14]:
df['Date'] = pd.to_datetime(df['Date'])            # convert the 'Date' column to a datetime object
df['Month-Year'] = df['Date'].dt.to_period('M')    # extract the month and year from the 'Date' column and create a new column 'Month-Year'
monthly_expenses_by_category = df.groupby(['Month-Year', 'Category'])['Debit'].sum().reset_index()

monthly_expenses_by_category['Debit']=monthly_expenses_by_category['Debit'].astype('str')
monthly_expenses_by_category['Debit']=monthly_expenses_by_category['Debit'].str.replace('-','')
monthly_expenses_by_category['Debit']=monthly_expenses_by_category['Debit'].astype('float')
monthly_expenses_by_category = monthly_expenses_by_category[monthly_expenses_by_category["Category"].str.contains("Credit") == False]

monthly_expenses_by_category = monthly_expenses_by_category.sort_values(by='Debit', ascending=False)
monthly_expenses_by_category['Debit'] = monthly_expenses_by_category['Debit'].round().astype(int)
monthly_expenses_by_category['Month-Year'] = monthly_expenses_by_category['Month-Year'].astype(str)
monthly_expenses_by_category = monthly_expenses_by_category.rename(columns={'Debit': 'Debit '})

monthly_expenses_by_category

,Month-Year,Category,Debit
25,2022-05,Giving,543799
45,2022-08,Giving,240435
57,2022-10,Entertainment,144000
32,2022-06,Giving,103402
104,2023-09,Self-care/Services,103000
86,2023-05,Giving,75926
115,2023-11,Self-care/Services,62550
99,2023-08,Self-care/Services,56000
51,2022-09,Giving,50027
75,2023-03,Giving,49275


In [15]:
first_category = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Airtime',
    'Self-care/Services',
    'Gov-charges',
    'Entertainment',
    'Giving',
    'Bank-Charges'
])
first_category


Select(options=['All', 'Airtime', ...], value='All')

In [18]:
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_by_category.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_by_category[monthly_expenses_by_category['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Debit ', yformatter='%0f').opts(xrotation=45) #use yformatter to format the y-axis.
    return plot

# define callback function
@pn.depends(first_category.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# create layout
monthly_expenses_by_category_chart = pn.Row(first_category, update_plot)
monthly_expenses_by_category_chart[1].width = 500

monthly_expenses_by_category_chart


Row
    [0] Select(options=['All', 'Airtime', ...], value='All')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=500)

*<font color= 'blue'><b> My Summary Table<b/><font/>*

In [19]:
df = df[['Date', 'Category', 'Description', 'Debit']]

# Convert 'Debit' column to string and replace empty strings with 0
df['Debit'] = df['Debit'].astype(str).replace('', '0', regex=True)

# Convert 'Debit' column to numeric and then to float
df['Debit'] = pd.to_numeric(df['Debit']).astype(float)

# Get absolute values
df['Debit'] = df['Debit'].abs()

# Exclude rows where 'Category' contains "Credit"
df = df[df["Category"].str.contains("Credit") == False]

# Round values and convert to int
df['Debit'] = df['Debit'].round().astype(int)
df



,Date,Category,Description,Debit
0,2022-01-01,Airtime,sbw/patu/1149225362/2136928210/2349024484050,300
1,2022-01-01,Airtime,sbw/patu/1149374657/2136928210/2349024484050,100
2,2022-01-03,Self-care/Services,pos pur @ 2kud1dyl-la la,2000
3,2022-01-05,Self-care/Services,pos pur @ 20570p9m-la la,2000
5,2022-01-09,Airtime,sbw/patu/1652500853/2136928210/2349024484050,300
6,2022-01-09,Self-care/Services,pos pur @ 2up1a787-teamapt limited moniepo472 203,3000
7,2022-01-09,Self-care/Services,pos pur @ 20507e29-ab ab,2350
8,2022-01-10,Gov-charges,fgn stamp duty/s78269630 on 09-jan-22 for account,50
9,2022-01-10,Self-care/Services,pos pur @ 2up1a787-teamapt limited moniepo075 207,3100
10,2022-01-12,Self-care/Services,pos pur @ 20507e29-ab ab,4700


In [20]:
def filter_df(category):
    """
    Description:
    This function filters a DataFrame based on the specified 'category' value. 
    If 'All' is provided, the original DataFrame is returned. Otherwise, 
    it filters the DataFrame to include only rows where the 'Category' column matches the given category.

    Parameters:
    - category (str): The category value to filter the DataFrame. Use 'All' to return the entire DataFrame.

    Returns:
    - pandas DataFrame: Filtered DataFrame based on the specified category.

    Example Usage:
    filtered_data = filter_df('Expense')
    all_data = filter_df('All')
    # Returns the original DataFrame since 'All' is specified
    """
    if category == 'All':
        return df
    return df[df['Category'] == category]

# Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=350)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget
first_category.param.watch(update_summary_table, 'value')

summary_table

DataFrame(height=300, sizing_mode='fixed', value=          Date            ..., width=350)

*<font color= 'blue'><b> Final Dashboard<b/><font/>*

In [24]:
template = pn.template.FastListTemplate(
    title="Personal Finances Summary",
    sidebar=[
        pn.pane.Markdown("## *Financial freedom is a journey, and wise money management is the compass that guides you to your destination.*"),
        pn.pane.PNG('https://i.imgur.com/PPuNGHl.png', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        first_category
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_by_category_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)

template.show()


Launching server at http://localhost:55723
